In [1]:
pip install -q -U beatrix-jupyterlab jupyterlab-lsp preprocessing llama-index llama-hub youtube_transcript_api transformers[torch] sentence-transformers accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cudf 23.8.0 requires pyarrow==11.*, but you have pyarrow 7.0.0 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2

In [2]:
import torch
from llama_hub.youtube_transcript import YoutubeTranscriptReader
from llama_index import PromptHelper, ServiceContext, VectorStoreIndex
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt
from transformers import AutoTokenizer, AutoModelForCausalLM

# Globals
model_name = "HuggingFaceH4/zephyr-7b-beta"
embedding_name = "BAAI/bge-small-en-v1.5"

# Youtube loader
new_heights_vids = ["https://www.youtube.com/watch?v=Zhv7aMcaCKM"]
loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=new_heights_vids)

# Prompts
system_prompt = (
    "<|SYSTEM|> You are a helpful, resourceful, and friendly "
    + "chatbot who always responds will well thought out answers."
)

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")
prompt_helper = PromptHelper(
    num_output=256,
    chunk_overlap_ratio=0.2,
)

In [3]:
import os

cache_dir = "cache_dir"
vector_db = "vector_db"

dirs = [cache_dir, vector_db]
for d in dirs:
    os.makedirs(d, exist_ok=True)

In [4]:
# Models
embedding = HuggingFaceEmbedding(embedding_name, cache_folder=cache_dir)

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta", device_map="auto", return_token_type_ids=False, use_fast=True)
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta", device_map="auto", torch_dtype=torch.bfloat16, cache_dir=cache_dir)

llm = HuggingFaceLLM(
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    context_window=3000,
    tokenizer=tokenizer,
    model=model,
#     tokenizer_name=tokenizer,
#     tokenizer_kwargs={"return_token_type_ids": False},
#     model_kwargs={"torch_dtype": torch.bfloat16, "cache_dir": cache_dir},
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.7,
        "top_k": 10,
        "top_p": 0.95,
    },
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
# Vector Store
service_context = ServiceContext.from_defaults(
    chunk_size=1024, llm=llm, embed_model=embedding, prompt_helper=prompt_helper
)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# persist
index.storage_context.persist("vector_db")

query_engine = index.as_query_engine()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
result = query_engine.query(
    "Summarize what Travis Kelce said about the Chiefs' loss to the Broncos."
)

In [13]:
print(result.response)


Travis Kelce acknowledged that the Chiefs' five turnovers in their loss to the Broncos was a problem and stated that they handed the Broncos 17 points off of turnovers and great field position, similar to what they were doing during the game. He also took responsibility for his team's performance and expressed determination to get better and correct their mistakes. Overall, Kelce's comments reflected a sense of accountability and urgency to address the team's turnovers and improve their performance.


In [14]:
halloween_result = query_engine.query(
    "What did they discuss about halloween?"
)

print(halloween_result.response)


The context information provided is a video where the hosts discuss Halloween-themed topics, such as their favorite Halloween candies, Halloween costumes they have seen, and Halloween-themed merchandise. They also mention a Halloween episode and a Halloween costume challenge for their viewers. Therefore, they discussed Halloween in detail during the video.


In [16]:
halloween2_result = query_engine.query(
    "What was Travis and Jason Kelce's favorite costume from there childhood?"
)

print(halloween2_result.response)


The context provided in the video does not include any information about Travis and Jason Kelce's favorite costume from their childhood. The video mainly focuses on their Halloween episode and discusses their favorite Halloween candies, costumes, and upcoming announcements.


In [17]:
query_engine.query(
    "What did Jason Kelce say about his game against the commanders. Give a summary."
).response

"\nThere is no information provided in the context about Jason Kelce's comments about his game against the commanders. Therefore, I am unable to summarize his statements."

In [20]:
print(query_engine.query("How would you summarize this episode of new heights? Provide a bulleted list of main points that I might find interesting. Include any breaking news.").response)


1. The episode is a Halloween-themed special, with the hosts discussing their favorite Halloween candies and conducting a candy taste test.
2. The hosts also reveal the official New Heights Halloween candy, chosen from suggestions by their followers.
3. The episode features a Halloween costume contest among the hosts and their families, with Jason's mother's baby in a Halloween costume receiving a lot of praise.
4. The hosts promote their podcast and YouTube channel, urging viewers to subscribe and follow them on social media.
5. The episode also features an announcement about the official New Heights fan club and a guest appearance by Jordan Mata on a special episode.
6. The hosts promote the Experienced Smart Money debit card, which builds credit without debt.
7. The episode includes some comedic moments, such as Travis's struggle to focus on the candy taste test and Jason's late arrival to a football game.
8. The hosts express their appreciation for their followers and listeners, t

In [ ]:
print()